# Intro

- Il problema del forecasting dei prezzi è un problema di regressione affrontato storicamente tramite i classici modelli di serie storiche ARMA-GARCH. Tali modelli consistono in regressioni lineari e, in quanto tali, hanno un bias significativo e che non sono in grado di gestire features categoriche.
- Stando alla letteratura "Assets Forecasting with Feature Engineering and Transformation Methods for LightGBM" (https://arxiv.org/abs/2501.07580) gli approcci più di Deep Neural Network più diffusi sono promettenti ma estremamente onerosi computazionalmente e quindi poco accessibili. Un ottimo compromesso per qualità delle previsioni ed efficienza computazionale è il modello LightGBM che verrà utilizzato in questa sfida.
- Per questa sfida ho scelto di utilizzare un indicatore del rischio geopolitico come fonte di dati alternativi, consapevole che sarebbe più opportuno utilizzare indicatori di sentiment daily basati sulle news intraday.

# Fonti dei dati

Ho scelto di utilizzare come base dati da cui estrarre features la serie storica dei prezzi e un indicatore del rischio geopolitico. Le fonti da cui ho scaricato i dati nella cartella data sono le seguenti:
- https://alfred.stlouisfed.org/series?seid=DCOILBRENTEU
- https://www.matteoiacoviello.com/gpr.htm

# Preprocessing

In [80]:
import pandas as pd

# caricamento della serie storica del prezzo del petrolio
brent_ts = pd.read_csv("data/vintages_starting_2025-06-18.csv")
brent_ts["observation_date"] = pd.to_datetime(brent_ts["observation_date"].astype(str), format="%Y-%m-%d")
gpr_ts = pd.read_excel("data/data_gpr_export.xls").loc[:,["month","GPR"]]
gpr_ts["month"] = pd.to_datetime(gpr_ts["month"].astype(str), format="%Y-%m-%d")

In [81]:
brent_ts.head(3)

,observation_date,DCOILBRENTEU_20250618
0,1987-05-20,18.63
1,1987-05-21,18.45
2,1987-05-22,18.55


In [82]:
# aggiungo a ciascuna riga il GPR del mese precedente
gpr_ts["prev_GPR"] = gpr_ts["GPR"].shift(1)

In [83]:
gpr_ts.tail()

,month,GPR,prev_GPR
1500,2025-01-01,112.333267,142.134567
1501,2025-02-01,136.192444,112.333267
1502,2025-03-01,175.432983,136.192444
1503,2025-04-01,141.136383,175.432983
1504,2025-05-01,158.005234,141.136383


In [84]:
gpr_ts.info()
brent_ts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1505 entries, 0 to 1504
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   month     1505 non-null   datetime64[ns]
 1   GPR       485 non-null    float64       
 2   prev_GPR  484 non-null    float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 35.4 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9934 entries, 0 to 9933
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   observation_date       9934 non-null   datetime64[ns]
 1   DCOILBRENTEU_20250618  9660 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 155.3 KB


In [85]:
df_dataset = pd.merge(
    left=brent_ts, 
    right=gpr_ts,
    how='left',
    left_on=['observation_date'],
    right_on=['month'],
).ffill().dropna(axis=0).rename(columns={"DCOILBRENTEU_20250618":"BrentPrice"})

In [86]:
df_dataset.head(30)

,observation_date,BrentPrice,month,GPR,prev_GPR
8,1987-06-01,18.65,1987-06-01,94.182404,98.650917
9,1987-06-02,18.68,1987-06-01,94.182404,98.650917
10,1987-06-03,18.75,1987-06-01,94.182404,98.650917
11,1987-06-04,18.78,1987-06-01,94.182404,98.650917
12,1987-06-05,18.65,1987-06-01,94.182404,98.650917
13,1987-06-08,18.75,1987-06-01,94.182404,98.650917
14,1987-06-09,18.78,1987-06-01,94.182404,98.650917
15,1987-06-10,18.78,1987-06-01,94.182404,98.650917
16,1987-06-11,18.68,1987-06-01,94.182404,98.650917
17,1987-06-12,18.78,1987-06-01,94.182404,98.650917


Ora calcoliamo le variazioni di prezzo a 1, 5 e 10 giorni

In [87]:
df_dataset["1d_BrentPrice"] = df_dataset["BrentPrice"].diff()
df_dataset["5d_BrentPrice"] = df_dataset["BrentPrice"].diff(5)
df_dataset["10d_BrentPrice"] = df_dataset["BrentPrice"].diff(10)

df_dataset = df_dataset.iloc[10 :,:]

In [88]:
df_dataset.head(30)

,observation_date,BrentPrice,month,GPR,prev_GPR,1d_BrentPrice,5d_BrentPrice,10d_BrentPrice
18,1987-06-15,18.78,1987-06-01,94.182404,98.650917,0.00,0.03,0.13
19,1987-06-16,18.90,1987-06-01,94.182404,98.650917,0.12,0.12,0.22
20,1987-06-17,19.03,1987-06-01,94.182404,98.650917,0.13,0.25,0.28
21,1987-06-18,19.05,1987-06-01,94.182404,98.650917,0.02,0.37,0.27
22,1987-06-19,19.05,1987-06-01,94.182404,98.650917,0.00,0.27,0.40
23,1987-06-22,19.10,1987-06-01,94.182404,98.650917,0.05,0.32,0.35
24,1987-06-23,18.90,1987-06-01,94.182404,98.650917,-0.20,0.00,0.12
25,1987-06-24,18.75,1987-06-01,94.182404,98.650917,-0.15,-0.28,-0.03
26,1987-06-25,18.70,1987-06-01,94.182404,98.650917,-0.05,-0.35,0.02
27,1987-06-26,19.08,1987-06-01,94.182404,98.650917,0.38,0.03,0.30
